In [1]:
import bw2io as bi # ensemble des fonctions et classes pour importer et exporter (input/output)
import bw2data as bd # ... pour gérer les données du projet
import bw2calc as bc # ... pour faire des opérations
import bw2analyzer as ba # ... pour interpréter les résultats
from pathlib import Path

import pandas as pd # pour utiliser un format de table pratique
import matplotlib.pyplot as plt

In [8]:
bi.backup.restore_project_directory(
    Path.cwd().joinpath(
        "brightway2_project_ecoinvent_311.tar.gz"
    ),
    overwrite_existing=True
)

Restoring project backup archive - this could take a few minutes...
Restored project: project_ecoinvent_311


'project_ecoinvent_311'

In [2]:
bd.projects.set_current("project_ecoinvent_311")
bd.projects.current

'project_ecoinvent_311'

In [10]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff

In [11]:
eidb = bd.Database('ecoinvent-3.11-cutoff')

In [14]:
activity = [
    a for a in eidb.search('heat production, air-water heat pump') \
        if 'Europe' in a['location']
][0]
activity

'heat production, air-water heat pump 10kW' (megajoule, Europe without Switzerland, None)

In [15]:
meth = [m for m in bd.methods if 'EF v3.1' in m[1] and 'no LT' not in m[1]]
gwp100 = [ind for ind in meth if 'GWP100' in str(ind) and 'biogenic' not in str(ind) and 'fossil' not in str(ind) and 'land' not in str(ind)][0]

In [16]:
lca = bc.LCA(demand={activity : 1.0}, method=gwp100) #Préparation de matrices réduites A, B, C
lca.lci()# Préparation du calcul de g. Si on appelle le score à cette étape, on obtient g.
lca.lcia() # Préparation du calcul de h.
lca.score

0.04136631331232376

In [19]:
from copy import deepcopy
one_MJ = deepcopy(lca.score)
one_MJ

0.04136631331232376

In [23]:
hvac_year = 80636
hvac_score = one_MJ * hvac_year * 3.6 # 1 kWh = 3.6 MJ
hvac_score

12008.210544909138

In [25]:
length = 54
width = 12
num_floors = 3
surf = length * width * num_floors
hvac_m2_score = hvac_score / surf
hvac_m2_score

6.1770630375047

In [27]:
hvac_m2 = hvac_year / surf
hvac_m2

41.47942386831276

In [28]:
280/6.2

45.16129032258064